In [1]:
import glob
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from openap import aero, nav

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader, DisjointLoader, BatchLoader
from spektral.layers import GATConv, DiffusionConv, GCNConv
from spektral.transforms import LayerPreprocess

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

import seaborn as sns

pd.set_option("display.max_columns", None)

tf.config.list_physical_devices("GPU")

2023-03-17 08:58:22.403019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 08:58:22.476468: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
start_year, end_year = 2016, 2019

start = datetime(start_year, 1, 1)
end = datetime(end_year, 12, 31)

agg_interval = 30  # minutes

## Load Data

In [3]:
files = sorted(glob.glob("data/airport/*.parquet"))

data = {}
airports = []

for file in tqdm(files):
    ap = Path(file).stem
    airports.append(ap)
    
    d =  pd.read_parquet(file).sort_values('timeslot')
    X = d.drop(["delay_arrival", "delay_departure"], axis=1)
    Y = d[["delay_arrival", "delay_departure"]].copy()
    T = d[["timeslot"]]
    
    data[ap] = {}
    data[ap]['X'] = X
    data[ap]['Y'] = Y
    data[ap]['T'] = T



100%|██████████| 50/50 [00:00<00:00, 82.56it/s]


In [4]:
# train_dates = d.query("date<'2019-01-01'").date.unique()
# test_dates = d.query("date>='2019-01-01'").date.unique()

train_dates = d.query("date<'2019-09-01'").date.unique()
test_dates = d.query("date>='2019-09-01'").date.unique()

In [5]:
X_all = pd.concat([d["X"] for ap, d in data.items()]).drop(["timeslot", "date"], axis=1)
Y_all = pd.concat([d["Y"] for ap, d in data.items()])

scaler = StandardScaler()
scaler.fit(pd.concat([X_all, Y_all], axis=1))


StandardScaler()

In [6]:
# corr = (pd.concat([X_all, Y_all], axis=1)).corr().round(2)

# plt.figure(figsize=(15, 15))
# plt.tight_layout()
# heat_map = sns.heatmap(
#     corr,
#     linewidth=1,
#     annot=True,
#     center=0,
#     cmap="Spectral_r",
#     vmin=-1,
#     vmax=1,
#     square=True,
#     # xticklabels=ticks,
#     # yticklabels=ticks,
# )
# # heat_map.set_xticklabels(ticks, rotation=30, ha="right")
# plt.show()


In [7]:
X_train = []
Y_train = []
X_test = []
Y_test = []


for airport in tqdm(airports):
    xy = pd.concat([data[airport]["X"], data[airport]["Y"]], axis=1)
    xy_train = xy.query("date in @train_dates")
    xy_test = xy.query("date in @test_dates")

    # train: size T x F
    x_train = xy_train.drop(columns=["timeslot", "date"])
    y_train = xy_train[["delay_arrival", "delay_departure"]]
    X_train.append(scaler.transform(x_train))
    Y_train.append(y_train)

    # test: size T x F
    x_test = xy_test.drop(columns=["timeslot", "date"])
    y_test = xy_test[["delay_arrival", "delay_departure"]]
    X_test.append(scaler.transform(x_test))
    Y_test.append(y_test)

X_train = np.stack(X_train)
Y_train = np.stack(Y_train)
X_test = np.stack(X_test)
Y_test = np.stack(Y_test)

# N x T x F
X_train = np.swapaxes(X_train, 0, 1)
Y_train = np.swapaxes(Y_train, 0, 1)
X_test = np.swapaxes(X_test, 0, 1)
Y_test = np.swapaxes(Y_test, 0, 1)

print(f"T x AP x F: X_train = {X_train.shape} | Y_train = {Y_train.shape}")
print(f"T x AP x F: X_test  = {X_test.shape} | Y_test  = {Y_test.shape}")


100%|██████████| 50/50 [00:00<00:00, 61.25it/s]


T x AP x F: X_train = (20496, 50, 27) | Y_train = (20496, 50, 2)
T x AP x F: X_test  = (2881, 50, 27) | Y_test  = (2881, 50, 2)


# Create tensorflow dataset

In [8]:
def createDataset(
    X: np.ndarray,
    Y: np.ndarray,
    lookback: int,
    lookahead: int,
    batch_size: int = 64,
):

    # make windows
    idx = 0
    X_ = []
    Y_ = []

    while idx + lookback + lookahead < len(X):
        # features
        # x = X[idx : idx + lookback, :, :]
        x = np.append(
            X[idx : idx + lookback, :, :], Y[idx : idx + lookback, :, :], axis=2
        )
        X_.append(x)

        # make labels with multi-horizon
        y = Y[idx + lookback : idx + lookback + lookahead, :, :]
        Y_.append(y)

        idx += 1

    X = np.array(X_)
    Y = np.array(Y_)

    # Ensure only complete batches are made (remove incomplete ones)
    batchCutoff = X.shape[0] - (X.shape[0] % batch_size)

    X, Y = (
        X[:batchCutoff, :, :, :],
        Y[:batchCutoff, :, :, :],
    )

    X = X.reshape(X.shape[0]*X.shape[2], X.shape[1], X.shape[3])
    Y = Y.reshape(Y.shape[0]*Y.shape[2], Y.shape[1], Y.shape[3])

    with tf.device("CPU"):
        dataset = tf.data.Dataset.from_tensor_slices((X, Y)).batch(batch_size)

    return dataset


In [9]:
epochs = 100
patience = 5
lookback = 8
lookahead = 8
batch_size = 64

In [10]:
train_dataset = createDataset(X_train, Y_train, lookback, lookahead, batch_size)
test_dataset = createDataset(X_test, Y_test, lookback, lookahead, batch_size)

print(train_dataset)
print(test_dataset)


2023-03-17 08:58:27.977674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 08:58:28.467431: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5937 MB memory:  -> device: 0, name: NVIDIA T1000 8GB, pci bus id: 0000:65:00.0, compute capability: 7.5


<BatchDataset element_spec=(TensorSpec(shape=(None, 8, 29), dtype=tf.float64, name=None), TensorSpec(shape=(None, 8, 2), dtype=tf.float64, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 8, 29), dtype=tf.float64, name=None), TensorSpec(shape=(None, 8, 2), dtype=tf.float64, name=None))>


## LSTM Model

In [11]:

lstm_units = 60

n_features = X_train.shape[2] + Y_train.shape[2]  # t-back to t0
n_targets = Y_train.shape[2]  # t0 to t+forward

n_nodes = len(airports)

print(n_features, n_targets, n_nodes)

29 2 50


In [12]:
input = layers.Input(
    shape=(lookback, n_features), batch_size=batch_size, name="Features"
)
print(input.shape)

lstm1 = layers.LSTM(lstm_units, dropout=0.1, return_sequences=True, name="LSTM1")(input)
lstm2 = layers.LSTM(lstm_units, dropout=0.1, return_sequences=False, name="LSTM2")(lstm1)


dense1 = layers.Dense(lstm_units, name="Dense1")(lstm2)
dense1 = tf.keras.layers.Dropout(0.1)(dense1)
print(dense1.shape)

dense2 = layers.Dense(n_targets * lookahead, name="DenseFinal")(dense1)
print(dense2.shape)

output = tf.reshape(
    dense2, (batch_size, lookahead, n_targets), name="ReshapeFinal"
)
print(output.shape)


(64, 8, 29)
(64, 60)
(64, 16)
(64, 8, 2)


In [13]:
model = keras.Model(inputs=input, outputs=output, name="LSTM")

model.compile(optimizer=Adam(), loss="mse")
# model.compile(optimizer=Adam(), loss="huber_loss")


model.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Features (InputLayer)       [(64, 8, 29)]             0         
                                                                 
 LSTM1 (LSTM)                (64, 8, 60)               21600     
                                                                 
 LSTM2 (LSTM)                (64, 60)                  29040     
                                                                 
 Dense1 (Dense)              (64, 60)                  3660      
                                                                 
 dropout (Dropout)           (64, 60)                  0         
                                                                 
 DenseFinal (Dense)          (64, 16)                  976       
                                                                 
 tf.reshape (TFOpLambda)     (64, 8, 2)                0      

In [14]:
es = keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[es],
    shuffle=False,
)


Epoch 1/100


2023-03-17 08:58:33.379448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-03-17 08:58:33.527122: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f4064025d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-17 08:58:33.527176: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA T1000 8GB, Compute Capability 7.5
2023-03-17 08:58:33.537670: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-17 08:58:33.607611: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-17 08:58:33.644827: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16000/16000 [==============================] - 123s 7ms/step - loss: 71.2404 - val_loss: 78.8526
Epoch 2/100
 4355/16000 [=======>......................] - ETA: 1:21 - loss: 68.1598

# Analysis


In [ ]:
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()


In [ ]:
Y_pred = model.predict(test_dataset)


## Generate output dataset

In [ ]:
buffer = []

for i, airport in enumerate(airports):
    xy = pd.concat([data[airport]["X"], data[airport]["Y"]], axis=1)
    xy_test = xy.query("date in @test_dates").iloc[lookback:]

    # Test delay metric, column represent lookahead time
    arr_delay = Y_test[lookback:, i, 0]
    n_row = len(arr_delay) - lookahead
    arr_delays = np.empty((n_row, lookahead))
    for k in range(lookahead):
        arr_delays[:, k] = arr_delay[k : k + n_row]

    dep_delay = Y_test[lookback:, i, 1]
    n_row = len(dep_delay) - lookahead
    dep_delays = np.empty((n_row, lookahead))
    for k in range(lookahead):
        dep_delays[:, k] = dep_delay[k : k + n_row]

    airport_delays = (
        pd.concat(
            [
                pd.DataFrame().assign(time=xy_test["timeslot"].values),
                pd.DataFrame(
                    arr_delays.round().astype(int),
                    columns=[f"arr_{30*n}m" for n in range(1, lookahead + 1)],
                ),
                pd.DataFrame(
                    dep_delays.round().astype(int),
                    columns=[f"dep_{30*n}m" for n in range(1, lookahead + 1)],
                ),
                pd.DataFrame(
                    Y_pred[:, :, i, 0].round().astype(int),
                    columns=[f"arr_est_{30*n}m" for n in range(1, lookahead + 1)],
                ),
                pd.DataFrame(
                    Y_pred[:, :, i, 1].round().astype(int),
                    columns=[f"dep_est_{30*n}m" for n in range(1, lookahead + 1)],
                ),
            ],
            axis=1,
        )
        .dropna()
        .assign(airport=airport)
    )

    buffer.append(airport_delays)

results = pd.concat(buffer, ignore_index=True)


In [ ]:
# results.to_csv("data/network_delay_gnn_results.csv", index=False)

In [ ]:
df = results.assign(time=lambda d: pd.to_datetime(d.time)).assign(
    date=lambda d: d.time.dt.date
)

res_arr = dict()
res_dep = dict()
for i in range(1, lookahead+1):
    res_arr[i] = {
        "Lookahead": f"{i*30} min",
        "MAE": mean_absolute_error(df[f"arr_{i*30}m"], df[f"arr_est_{i*30}m"]),
        "RMSE": mean_squared_error(
            df[f"arr_{i*30}m"], df[f"arr_est_{i*30}m"], squared=False
        ),
        "R2": r2_score(df[f"arr_{i*30}m"], df[f"arr_est_{i*30}m"]),
    }
    res_dep[i] = {
        "Lookahead": f"{i*30} min",
        "MAE": mean_absolute_error(df[f"dep_{i*30}m"], df[f"dep_est_{i*30}m"]),
        "RMSE": mean_squared_error(
            df[f"dep_{i*30}m"], df[f"dep_est_{i*30}m"], squared=False
        ),
        "R2": r2_score(df[f"dep_{i*30}m"], df[f"dep_est_{i*30}m"]),
    }

stats_arr = pd.DataFrame.from_dict(res_arr, orient="index")
stats_dep = pd.DataFrame.from_dict(res_dep, orient="index")

display(stats_arr)
display(stats_dep)


In [ ]:
def plot_delay(airport):
    df = results.query('airport == @airport')

    fig, axes = plt.subplots(5, 1, figsize=(20, 8), sharex=True, sharey=True)

    axes[0].set_title(f"{airport} - Arrival delay (min)")

    ax = axes[0]
    ax.plot(df.arr_30m, alpha=0.9, label="Actual")
    ax.plot(df.arr_est_30m, color="r", alpha=0.8, label="Predicted (-30 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)
    ax.set_ylim(-20, 80)

    ax = axes[1]
    ax.plot(df.arr_60m, alpha=0.9, label="Actual")
    ax.plot(df.arr_est_60m, color="r", alpha=0.8, label="Predicted (-60 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    ax = axes[2]
    ax.plot(df.arr_120m, alpha=0.9, label="Actual")
    ax.plot(df.arr_est_120m, color="r", alpha=0.8, label="Predicted (-120 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    ax = axes[3]
    ax.plot(df.arr_180m, alpha=0.9, label="Actual")
    ax.plot(df.arr_est_180m, color="r", alpha=0.8, label="Predicted (-180 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    ax = axes[4]
    ax.plot(df.arr_240m, alpha=0.9, label="Actual")
    ax.plot(df.arr_est_240m, color="r", alpha=0.8, label="Predicted (-240 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    # ax = axes[5]
    # ax.plot(df.arr_300m, alpha=0.9, label="Actual")
    # ax.plot(df.arr_est_300m, color="r", alpha=0.8, label="Predicted (-300 min)")
    # ax.axhline(0, ls=":")
    # ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    plt.tight_layout()
    plt.show()

    fig, axes = plt.subplots(5, 1, figsize=(20, 8), sharex=True, sharey=True)

    axes[0].set_title(f"{airport} - Departure delay (min)")

    ax = axes[0]
    ax.plot(df.dep_30m, alpha=0.9, label="Actual")
    ax.plot(df.dep_est_30m, color="r", alpha=0.8, label="Predicted (-30 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)
    ax.set_ylim(-20, 40)

    ax = axes[1]
    ax.plot(df.dep_60m, alpha=0.9, label="Actual")
    ax.plot(df.dep_est_60m, color="r", alpha=0.8, label="Predicted (-60 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    ax = axes[2]
    ax.plot(df.dep_120m, alpha=0.9, label="Actual")
    ax.plot(df.dep_est_120m, color="r", alpha=0.8, label="Predicted (-120 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    ax = axes[3]
    ax.plot(df.dep_180m, alpha=0.9, label="Actual")
    ax.plot(df.dep_est_180m, color="r", alpha=0.8, label="Predicted (-180 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    ax = axes[4]
    ax.plot(df.dep_240m, alpha=0.9, label="Actual")
    ax.plot(df.dep_est_240m, color="r", alpha=0.8, label="Predicted (-240 min)")
    ax.axhline(0, ls=":")
    ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    # ax = axes[5]
    # ax.plot(df.dep_300m, alpha=0.9, label="Actual")
    # ax.plot(df.dep_est_300m, color="r", alpha=0.8, label="Predicted (-300 min)")
    # ax.axhline(0, ls=":")
    # ax.legend(loc="upper left", borderaxespad=0.2, ncol=2)

    plt.tight_layout()
    plt.show()



In [ ]:
plot_delay("EHAM")